In [7]:
# Just to test at the very begining
print("Hello world")

Hello world


In [1]:
import numpy as np
import IPythonMagic
from Timer import Timer
import matplotlib as plt
import pycuda.driver as cuda_driver
import pycuda.compiler as cuda_compiler
from pycuda.gpuarray import GPUArray


In [2]:
%setup_logging

Python version 3.6.6 (default, Sep 12 2018, 18:26:19) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]


In [3]:
%cuda_context_handler context

Registering context in user workspace
Creating context
PyCUDA version 2018.1.1
CUDA version (9, 1, 0)
Driver version 10000
Using 'Tesla K80' GPU
 => compute capability: (3, 7)
 => memory: 11002 / 11441 MB available
Created context handle <43141776>
Using CUDA cache dir /home/ubuntu/jupyter_notebooks/Thomas_Bellotti/MilanoGPU2018/notebooks/cuda_cache


In [14]:
kernel_src = """

__global__ void shmemReduction(float * output, float * input, int size){

    // First we stride through global memory and compute the 
    // maximum for every thread
    
    int gid = blockIdx.x * blockDim.x + threadIdx.x;
    
    float max_value = -999999.99; // FIx me ..
    
    // threadIdx.x is always zero since we use one block.
    for (int i = threadIdx.x; i < size; i = i + blockDim.x){
        max_value = fmaxf(max_value, input[i]); 
    }
    
    // Temporary ouutput to check
    
    output[threadIdx.x] = max_value; 
    
    
    // Store the pair thread maximum in shared memory
    
    __shared__ float max_shared[32];
    max_shared[threadIdx.x] = max_value;
    
    // Synchronize in order all the thread to see the same shared memory
    
    __syncthreads();
    
    // Find max in shared memory
    // Reduce from 32 to 16 elements
    if (threadIdx.x < 16){
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 16]);
    }
     // Reduce from 16 to 8 elements
    if (threadIdx.x < 8){
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 8]);
    }
    // Reduce from 8 to 4 elements
    if (threadIdx.x < 4){
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 4]);
    }
    // Reduce from 4 to 2 elements
    if (threadIdx.x < 2){
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 2]);
    }
    // Reduce from 2 to 1 elements
    if (threadIdx.x < 1){
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 1]);
    }
    
    // Write out to output
    
    if (threadIdx.x == 0)
        output[0] = max_shared[0];
}

"""

kernel_module = cuda_compiler.SourceModule (kernel_src)
kernel_function = kernel_module.get_function("shmemReduction")

/home/ubuntu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:61: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(9): warning: variable "gid" was declared but never referenced




In [15]:
n = 64
a = np.random.random((1,n)).astype(np.float32)


a_g = GPUArray(a.shape, a.dtype)
a_g.set(a)

num_threads = 32
b = np.empty((11,num_threads)).astype(np.float32)

b_g = GPUArray (b.shape, b.dtype)

In [20]:
block_size = (num_threads, 1, 1)
grid_size = (1,1,1)

kernel_function(b_g, a_g, np.int32(n), grid = grid_size, block = block_size)

b_g.get(b)

print(b[0])

print(np.max(a))

[0.9863307  0.45503062 0.8536101  0.72530013 0.8314264  0.5278172
 0.48564497 0.96148294 0.4121069  0.5963757  0.7839482  0.91644627
 0.27043173 0.70218146 0.48154196 0.63125986 0.75924814 0.5139514
 0.44183996 0.3956442  0.9863307  0.45015293 0.5160185  0.3158753
 0.9843556  0.60835934 0.74241364 0.52127564 0.84744126 0.75101244
 0.50365454 0.74133193]
0.9863307
